In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport sklearn.metrics as metrics

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("/home/chuyu/fish_place_cell_paper/functions/func_map.jl")
include("/home/chuyu/fish_place_cell_paper/functions/func_stat.jl")
include("/home/chuyu/fish_place_cell_paper/functions/func_data.jl")
include("/home/chuyu/fish_place_cell_paper/functions/func_plot.jl")
include("/home/chuyu/fish_place_cell_paper/functions/utils.jl")

# Example fish

In [ ]:
experiment_filename_1 = "20230227_155813"
server_1 = 1

experiment_filename_2 = "20230227_165253"
server_2 = 1

experiment_filename_3 = "20230227_175948"
server_3 = 1

experimenter = "chuyu"
analyzer = "chuyu"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

ds_save_3 = Dataset(experiment_filename_3, experimenter, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_3 = Dataset(experiment_filename_3, "chuyu", gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")
ds_save_analyzer_3 = Dataset(experiment_filename_3, analyzer, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")

In [ ]:

NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
A_dF_1 = HDF5.readmmap(NMF_file["A_dF"])
close(NMF_file)

NMF_filename = joinpath(data_path(ds_save_cy_2), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
A_dF_2 = HDF5.readmmap(NMF_file["A_dF"])
close(NMF_file)

NMF_filename = joinpath(data_path(ds_save_cy_3), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
A_dF_3 = HDF5.readmmap(NMF_file["A_dF"])
close(NMF_file)

n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end



img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

    
global img_bg_end_1 = img_bg_1[:,:,end]


global w = size(img_bg_end_1, 1)
global l = size(img_bg_end_1, 2)
    

for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n100.h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
close(for_place_calculation_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
global countour_array_2 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_3), "chamber_geometry_$(experiment_filename_3).h5"))
chamber_roi_3 = read(orientation_correction_file,"chamber_roi")
global countour_array_3 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)

In [ ]:
all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][2]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
close(file)

place_cell_index_1 = intersect(place_cell_index_1, findall(whether_tel))
    
all_files = readdir(data_path(ds_save_cy_2))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
specificity_2 = HDF5.readmmap(file["specificity"])
specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
close(file)


all_files = readdir(data_path(ds_save_cy_3))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_3, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_3), save_file_name)
file = h5open(info_filename, "r")
place_map_all_3 = HDF5.readmmap(file["place_map_all"])
specificity_3 = HDF5.readmmap(file["specificity"])
specificity_population_z_3 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_3 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_3 = HDF5.readmmap(file["valid_neurons"])
close(file)

In [ ]:
bvc_filename = joinpath(data_path(ds_save_cy_1), "BVC_tel_newbound.h5")
bvc_file = h5open(bvc_filename, "r")
corr_diff = read(bvc_file, "corr_diff")
popt_all_1 = read(bvc_file, "popt_all_1")
popt_all_2 = read(bvc_file, "popt_all_2")
popt_all_3 = read(bvc_file, "popt_all_3")
candidate_neurons = read(bvc_file, "candidate_neurons") .+1
predict_map_all_1 = read(bvc_file, "predict_map_all_1")
predict_map_12_all = read(bvc_file, "predict_map_12_all")
close(bvc_file)

In [ ]:
d_1 = popt_all_1[1, candidate_neurons]
d_2 = popt_all_2[1, candidate_neurons]
d_3 = popt_all_3[1, candidate_neurons]

Φ_1 = popt_all_1[2, candidate_neurons]
Φ_2 = popt_all_2[2, candidate_neurons]
Φ_3 = popt_all_3[2, candidate_neurons]



Φ_threshold = sqrt(2)/2


Φ_pass = (numpy.cos(Φ_1) .< -Φ_threshold).*(numpy.cos(Φ_3) .< -Φ_threshold)
pass_neurons_1 = candidate_neurons[Φ_pass]
Φ_pass = (numpy.cos(Φ_1) .> Φ_threshold).*(numpy.cos(Φ_3) .> Φ_threshold)
pass_neurons_2 = candidate_neurons[Φ_pass]
pass_neurons = union(pass_neurons_1, pass_neurons_2);

In [ ]:
function map_corr_p_value(which_map_1, which_map_2)
    mask = findall(isfinite.(which_map_1) .* isfinite.(which_map_2))
    n_shuffle = length(mask) - 1
    random_map_cov_distribution = fill(NaN32, n_shuffle)

    map_1_values = which_map_1[mask]
    map_2_values = which_map_2[mask]
    map_cov = cov(map_1_values, map_2_values)
    for i in 1:n_shuffle
        shift = i
        map_1_values_shuffle = numpy.roll(map_1_values, shift)
        random_map_cov_distribution[i] = cov(map_1_values_shuffle, map_2_values)
    end
    p_value = sum(random_map_cov_distribution.>map_cov)./length(random_map_cov_distribution)
    return p_value
end

In [ ]:
pvalue_map_1_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_1[:,:,which_neuron];
    pvalue_map_1_without_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end

pvalue_map_1_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_1[:,:,which_neuron];
    pvalue_map_1_with_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end


pvalue_map_2_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    pvalue_map_2_without_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end

pvalue_map_2_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    pvalue_map_2_with_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end

pvalue_map_3_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_3[:,:,which_neuron];
    pvalue_map_3_without_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end

pvalue_map_3_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_3[:,:,which_neuron];
    pvalue_map_3_with_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end


pvalue_map_2_half1_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron = candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];

    shift = 0
    map1 = which_map_1[55- shift:93 - shift,:];
    map2 = which_map_2[55- shift:93 - shift,:];
    
    pvalue_map_2_half1_with_wall[which_neuron] = map_corr_p_value(map1, map2)
end


pvalue_map_2_half2_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron = candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];

    shift = 40
    map1 = which_map_1[55- shift:93 - shift,:];
    map2 = which_map_2[55- shift:93 - shift,:];
    
    pvalue_map_2_half2_with_wall[which_neuron] = map_corr_p_value(map1, map2)
end

In [ ]:
pvalue_13 = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron = candidate_neurons
    map1 = place_map_all_1[:,:,which_neuron];
    map2 = place_map_all_3[:,:,which_neuron];

    pvalue_13[which_neuron] = map_corr_p_value(map1, map2)
end

In [ ]:
prediction_map_1_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_1[:,:,which_neuron];
    prediction_map_1_without_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

prediction_map_1_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_1[:,:,which_neuron];
    prediction_map_1_with_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end


prediction_map_2_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    prediction_map_2_without_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

prediction_map_2_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    prediction_map_2_with_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

prediction_map_3_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_3[:,:,which_neuron];
    prediction_map_3_without_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

prediction_map_3_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_3[:,:,which_neuron];
    prediction_map_3_with_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

prediction_map_2_half1_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron = candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];

    shift = 0
    map1 = which_map_1[55- shift:93 - shift,:];
    map2 = which_map_2[55- shift:93 - shift,:];
    
    prediction_map_2_half1_with_wall[which_neuron] = corr_2d(map1, map2)
end


prediction_map_2_half2_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron = candidate_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];

    shift = 40
    map1 = which_map_1[55- shift:93 - shift,:];
    map2 = which_map_2[55- shift:93 - shift,:];
    
    prediction_map_2_half2_with_wall[which_neuron] = corr_2d(map1, map2)
end

In [ ]:
@pyimport sklearn.metrics as metrics

function find_best_peak(which_map;threshold=8/10,components_size_threshold=20)

    components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = components_size_threshold)

    peak_values = [numpy.nanpercentile(which_map[img_label_valid .== valid_components[i]], 95) for i in 1:length(valid_components)]
    peak_weighted_areas = [numpy.nansum(which_map[img_label_valid .== valid_components[i]]) for i in 1:length(valid_components)]


    best_peak = NaN32
    if length(peak_values) .> 0
        best_peak = components_peaks[findmax(peak_values)[2]]

    end

    return best_peak
end

In [ ]:
min_peak_shift_13 = fill(NaN32, n_neuron)
@showprogress for neuron_idx in  pass_neurons
    map_1 = copy(place_map_all_1[:,:, neuron_idx])
    map_3 = copy(place_map_all_3[:,:, neuron_idx])

    components_peaks_1, _, valid_components_1 = map_components_peak(map_1, components_size_threshold = 20)
    components_peaks_2, _, valid_components_2 = map_components_peak(map_3, components_size_threshold = 20)


    pairwise_distances = metrics.pairwise_distances(components_peaks_1, components_peaks_2)
    min_distance = minimum(pairwise_distances)
    min_peak_shift_13[neuron_idx] = min_distance
end

low_peak_shift_neuron = findall(min_peak_shift_13.<15);

In [ ]:
candidate_neuron = intersect(pass_neurons, findall(pvalue_map_1_without_wall.<0.05), findall(prediction_map_2_with_wall.>prediction_map_2_without_wall), findall(pvalue_map_2_with_wall.<0.05), findall(pvalue_13 .<0.05))

BVC_score = (prediction_map_2_with_wall .- prediction_map_2_without_wall)
candidate_neuron = candidate_neuron[sortperm(BVC_score[candidate_neuron];rev=true)]

In [ ]:
plot_bvc_cell = [51921,39217, 17206, 10414, 3869, 19508, 52748, 14486, 36375, 4960]

In [ ]:
plot_nonbvc_cell = [42235,40871,37314,1183, 21944, 36012, 46070, 14322, 2841, 7796]

In [ ]:
pvalue_map_1_without_wall[plot_bvc_cell]

In [ ]:
pvalue_map_2_with_wall[plot_bvc_cell]

In [ ]:
pvalue_map_3_without_wall[plot_bvc_cell]

In [ ]:
pvalue_13[plot_bvc_cell]

In [ ]:
valid_frame = isfinite.(place_map_all_1[:,:,1]); 
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-1;
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+1;
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-1;
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+1;

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/review_figures/bvc"

In [ ]:
which_neuron = plot_bvc_cell

fig = figure(figsize = (40,5*ceil(Int64, length(which_neuron)/10)), dpi=250)

which_map = place_map_all_1[:,:,which_neuron];    
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10, i+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', cmap = "jet") 

    axis("off")
    cell_index = which_neuron[i]
    score = numpy.round(corr_diff[cell_index],3)
    # title(cell_index)
    xlim(xmin,xmax)
    ylim(ymax,ymin)
end

which_map = place_map_all_2[:,:,which_neuron];    
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10,i+10+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', cmap = "jet") 

    axis("off")
    cell_index = which_neuron[i]
    # title("$(cell_index)" )
    xlim(xmin,xmax)
    ylim(ymax,ymin)
end


which_map = place_map_all_3[:,:,which_neuron];    
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10,i+20+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', cmap = "jet") 

    axis("off")
    cell_index = which_neuron[i]
    # title("$(cell_index)" )
    xlim(xmin,xmax)
    ylim(ymax,ymin)
end


subplots_adjust(left = 0, top = 0.1, right = 0.2, bottom = 0)


fig.savefig(joinpath(fig_path, "bvc_examples.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
which_neuron = plot_nonbvc_cell

fig = figure(figsize = (40,5*ceil(Int64, length(which_neuron)/10)), dpi=250)

which_map = place_map_all_1[:,:,which_neuron];    
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10, i+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', cmap = "jet") 

    axis("off")
    cell_index = which_neuron[i]
    score = numpy.round(corr_diff[cell_index],3)
    # title(cell_index)
    xlim(xmin,xmax)
    ylim(ymax,ymin)
end

which_map = place_map_all_2[:,:,which_neuron];    
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10,i+10+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', cmap = "jet") 

    axis("off")
    cell_index = which_neuron[i]
    # title("$(cell_index)" )
    xlim(xmin,xmax)
    ylim(ymax,ymin)
end


which_map = place_map_all_3[:,:,which_neuron];    
for i in 1:size(which_map,3)
    subplot(3*ceil(Int64, size(which_map,3)/10), 10,i+20+20*floor(Int32, (i-1)/10))
    imshow(which_map[:,:,i]', cmap = "jet") 

    axis("off")
    cell_index = which_neuron[i]
    # title("$(cell_index)" )
    xlim(xmin,xmax)
    ylim(ymax,ymin)
end


subplots_adjust(left = 0, top = 0.1, right = 0.2, bottom = 0)


fig.savefig(joinpath(fig_path, "non_bvc_examples.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

# Percentage for all fish

In [ ]:
data_info_all = 

[
    ["20230301_175518", 7, "20230301_191802", 7, "20230301_202336", 7, "chuyu"],
    ["20230227_155813", 1, "20230227_165253", 1, "20230227_175948", 1, "chuyu"],
    ["20230301_123207", 8, "20230301_132953", 8, "20230301_142431", 8, "chuyu"],
]

In [ ]:
BVC_percentage_all = []
BVC_list = []
for which_data in 1:length(data_info_all)
data_info = data_info_all[which_data]
experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experiment_filename_3 = data_info[5]
server_3 = data_info[6]

experimenter = "chuyu"
analyzer = "chuyu"


ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

ds_save_3 = Dataset(experiment_filename_3, experimenter, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_3 = Dataset(experiment_filename_3, "chuyu", gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")
ds_save_analyzer_3 = Dataset(experiment_filename_3, analyzer, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")



all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][2]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
close(file)
    
    
all_files = readdir(data_path(ds_save_cy_2))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
specificity_2 = HDF5.readmmap(file["specificity"])
specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
close(file)


all_files = readdir(data_path(ds_save_cy_3))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_3, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_3), save_file_name)
file = h5open(info_filename, "r")
place_map_all_3 = HDF5.readmmap(file["place_map_all"])
specificity_3 = HDF5.readmmap(file["specificity"])
specificity_population_z_3 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_3 = HDF5.readmmap(file["specificity_shuffle_z"])
valid_roi_3 = HDF5.readmmap(file["valid_neurons"])
close(file)


bvc_filename = joinpath(data_path(ds_save_cy_1), "BVC_tel_newbound.h5")
bvc_file = h5open(bvc_filename, "r")
corr_diff = read(bvc_file, "corr_diff")
popt_all_1 = read(bvc_file, "popt_all_1")
popt_all_2 = read(bvc_file, "popt_all_2")
popt_all_3 = read(bvc_file, "popt_all_3")
candidate_neurons = read(bvc_file, "candidate_neurons") .+1
predict_map_all_1 = read(bvc_file, "predict_map_all_1")
predict_map_12_all = read(bvc_file, "predict_map_12_all")
close(bvc_file)


d_1 = popt_all_1[1, candidate_neurons]
d_2 = popt_all_2[1, candidate_neurons]
d_3 = popt_all_3[1, candidate_neurons]

Φ_1 = popt_all_1[2, candidate_neurons]
Φ_2 = popt_all_2[2, candidate_neurons]
Φ_3 = popt_all_3[2, candidate_neurons]



Φ_threshold = sqrt(2)/2


Φ_pass = (numpy.cos(Φ_1) .< -Φ_threshold).*(numpy.cos(Φ_3) .< -Φ_threshold)
pass_neurons_1 = candidate_neurons[Φ_pass]
Φ_pass = (numpy.cos(Φ_1) .> Φ_threshold).*(numpy.cos(Φ_3) .> Φ_threshold)
pass_neurons_2 = candidate_neurons[Φ_pass]
pass_neurons = union(pass_neurons_1, pass_neurons_2);


pvalue_map_1_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in pass_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_1[:,:,which_neuron];
    pvalue_map_1_without_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end

pvalue_map_2_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in pass_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    pvalue_map_2_with_wall[which_neuron] = map_corr_p_value(which_map_1, which_map_2)
end


prediction_map_2_without_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in pass_neurons
    which_map_1 = predict_map_all_1[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    prediction_map_2_without_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

prediction_map_2_with_wall = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron in pass_neurons
    which_map_1 = predict_map_12_all[:,:,which_neuron];
    which_map_2 = place_map_all_2[:,:,which_neuron];
    prediction_map_2_with_wall[which_neuron] = corr_2d(which_map_1, which_map_2)
end

pvalue_13 = fill(NaN32, size(place_map_all_1,3))
@showprogress for which_neuron = pass_neurons
    map1 = place_map_all_1[:,:,which_neuron];
    map2 = place_map_all_3[:,:,which_neuron];

    pvalue_13[which_neuron] = map_corr_p_value(map1, map2)
end   
    
BVC_neuron = intersect(pass_neurons, findall(pvalue_map_1_without_wall.<0.05), findall(prediction_map_2_with_wall.>prediction_map_2_without_wall), findall(pvalue_map_2_with_wall.<0.05), findall(pvalue_13 .<0.05))
println(length(BVC_neuron))

    
BVC_score = (prediction_map_2_with_wall .- prediction_map_2_without_wall)
BVC_neuron = BVC_neuron[sortperm(BVC_score[BVC_neuron];rev=true)]
BVC_percentage = length(BVC_neuron)./length(pass_neurons)


println(BVC_percentage)
    
append!(BVC_percentage_all, BVC_percentage)
append!(BVC_list, [BVC_neuron])
end

In [ ]:
PC_list = []
tel_neuron_list = []
for which_data in 1:length(data_info_all)
data_info = data_info_all[which_data]
experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experiment_filename_3 = data_info[5]
server_3 = data_info[6]

experimenter = "chuyu"
analyzer = "chuyu"


ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

ds_save_3 = Dataset(experiment_filename_3, experimenter, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_3 = Dataset(experiment_filename_3, "chuyu", gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")
ds_save_analyzer_3 = Dataset(experiment_filename_3, analyzer, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")


NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
A_dF_1 = HDF5.readmmap(NMF_file["A_dF"])
close(NMF_file)


n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end



all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][2]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
close(file)
place_cell_index_1 = intersect(place_cell_index_1, findall(whether_tel))

    
    
all_files = readdir(data_path(ds_save_cy_2))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
specificity_2 = HDF5.readmmap(file["specificity"])
specificity_population_z_2 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_2 = HDF5.readmmap(file["specificity_shuffle_z"])
close(file)


all_files = readdir(data_path(ds_save_cy_3))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_3, candidate_filename[i]) for i in 1:length(candidate_filename)]
save_file_name = candidate_filename[which_file][1]
println(save_file_name)
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(data_path(ds_save_cy_3), save_file_name)
file = h5open(info_filename, "r")
place_map_all_3 = HDF5.readmmap(file["place_map_all"])
specificity_3 = HDF5.readmmap(file["specificity"])
specificity_population_z_3 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_3 = HDF5.readmmap(file["specificity_shuffle_z"])
close(file)


valid_roi_1 = findall(isfinite.(specificity_1))
valid_roi_2 = findall(isfinite.(specificity_2))
valid_roi_3 = findall(isfinite.(specificity_3))

append!(PC_list, [intersect(place_cell_index_1, valid_roi_1, valid_roi_2, valid_roi_3)])
append!(tel_neuron_list, [intersect(findall(whether_tel), valid_roi_1, valid_roi_2, valid_roi_3)])
end

In [ ]:
pass_neurons_all = []

for which_data in 1:length(data_info_all)
data_info = data_info_all[which_data]
experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experiment_filename_3 = data_info[5]
server_3 = data_info[6]

experimenter = "chuyu"
analyzer = "chuyu"


ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

ds_save_3 = Dataset(experiment_filename_3, experimenter, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_3 = Dataset(experiment_filename_3, "chuyu", gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")
ds_save_analyzer_3 = Dataset(experiment_filename_3, analyzer, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")



bvc_filename = joinpath(data_path(ds_save_cy_1), "BVC_tel_newbound.h5")
bvc_file = h5open(bvc_filename, "r")
corr_diff = read(bvc_file, "corr_diff")
popt_all_1 = read(bvc_file, "popt_all_1")
popt_all_2 = read(bvc_file, "popt_all_2")
popt_all_3 = read(bvc_file, "popt_all_3")
candidate_neurons = read(bvc_file, "candidate_neurons") .+1
predict_map_all_1 = read(bvc_file, "predict_map_all_1")
predict_map_12_all = read(bvc_file, "predict_map_12_all")
close(bvc_file)


d_1 = popt_all_1[1, candidate_neurons]
d_2 = popt_all_2[1, candidate_neurons]
d_3 = popt_all_3[1, candidate_neurons]

Φ_1 = popt_all_1[2, candidate_neurons]
Φ_2 = popt_all_2[2, candidate_neurons]
Φ_3 = popt_all_3[2, candidate_neurons]



Φ_threshold = sqrt(2)/2


Φ_pass = (numpy.cos(Φ_1) .< -Φ_threshold).*(numpy.cos(Φ_3) .< -Φ_threshold)
pass_neurons_1 = candidate_neurons[Φ_pass]
Φ_pass = (numpy.cos(Φ_1) .> Φ_threshold).*(numpy.cos(Φ_3) .> Φ_threshold)
pass_neurons_2 = candidate_neurons[Φ_pass]
pass_neurons = union(pass_neurons_1, pass_neurons_2);
append!(pass_neurons_all, [pass_neurons])



end

# BVC percenttage

In [ ]:
BVC_percentage_all

In [ ]:
mean(BVC_percentage_all) .* 100

In [ ]:
std(BVC_percentage_all).* 100

# Candidate length

In [ ]:
[length(x) for x in pass_neurons_all]

In [ ]:
mean([length(x) for x in pass_neurons_all])

In [ ]:
std([length(x) for x in pass_neurons_all])

# BVC in candidate non PC

In [ ]:
for i in 1:length(pass_neurons_all)
    println((length(BVC_list[i]) - length(intersect(PC_list[i], BVC_list[i])))/(length(pass_neurons_all[i]) - length(intersect(PC_list[i], pass_neurons_all[i]))))
end

In [ ]:
mean([
0.04734073641145529
0.050724637681159424
0.048280907095830286
]) * 100

In [ ]:
std([
0.04734073641145529
0.050724637681159424
0.048280907095830286
] .* 100) 

# BVC in candidate PC

In [ ]:
for i in 1:length(pass_neurons_all)
    println(length(intersect(PC_list[i], BVC_list[i]))/length(intersect(PC_list[i], pass_neurons_all[i])))
end

In [ ]:
mean([
0.03783783783783784
0.04375
0.0425531914893617
]) * 100

In [ ]:
std([
0.03783783783783784
0.04375
0.0425531914893617
] .* 100) 

# BVC in PC (old)

In [ ]:
for i in 1:length(pass_neurons_all)
    println(length(intersect(PC_list[i], BVC_list[i]))/length(intersect(PC_list[i])))
end

In [ ]:
mean([
0.016055045871559634
0.02626641651031895
0.023980815347721823
]) * 100

In [ ]:
std([
0.016055045871559634
0.02626641651031895
0.023980815347721823
] .* 100) 

# candidate in PC

In [ ]:
for i in 1:length(pass_neurons_all)
    println(length(intersect(PC_list[i], pass_neurons_all[i]))/length(intersect(PC_list[i])))
end

In [ ]:
mean([
0.4243119266055046
0.600375234521576
0.5635491606714629
].*100)

In [ ]:
std([
0.4243119266055046
0.600375234521576
0.5635491606714629
].*100)

In [ ]:
# candidate PC number
for i in 1:length(pass_neurons_all)
    println(length(intersect(PC_list[i], pass_neurons_all[i])))
end

In [ ]:
mean([
185
320
235
])

In [ ]:
# candidate non-PC number

for i in 1:length(pass_neurons_all)
    println(length(pass_neurons_all[i]) - length(intersect(PC_list[i], pass_neurons_all[i])))
end

In [ ]:
mean([
1711
1104
1367
])

# candidate in Tel

In [ ]:
for i in 1:length(pass_neurons_all)
    println(length(intersect(tel_neuron_list[i], pass_neurons_all[i]))/length(intersect(tel_neuron_list[i])))
end

In [ ]:
mean([
0.25656292286874155
0.24442155853072434
0.1958914159941306
].*100)

In [ ]:
std([
0.25656292286874155
0.24442155853072434
0.1958914159941306
].*100)

# Tel number

In [ ]:
for i in 1:length(pass_neurons_all)
    println(length(tel_neuron_list[i]))
end

In [ ]:
mean([
7390
5826
8178
])

In [ ]:
std([
7390
5826
8178
])

# Tel non candidate number

In [ ]:
for i in 1:length(pass_neurons_all)
    println(length(tel_neuron_list[i]) - length(pass_neurons_all[i]))
end

In [ ]:
mean([
5494
4402
6576

])

# BVC number

In [ ]:
for i in 1:length(pass_neurons_all)
    println((length(BVC_list[i])))
end

In [ ]:
mean([88
70
76])